In [0]:
# packages used
import os
from datetime import datetime
import json
from pyspark.sql.types import StructType, StructField, DateType, StringType, TimestampType, IntegerType, FloatType

In [0]:
# Azure credentials
storageAccountName = 'saderekguidedcapstone'
storageAccountAccessKey = 'rBDfcjXMi5ZBFivixha63KnAInUtkjMYUsPeZmV+aKXZIfHN03N/nrv6Q63jOQaIOpM3nWLRd1twLjaPzzQcTA=='
blobContainerName = 'container1'

In [0]:
# make files in Azure accessible to PySpark
if not any(mount.mountPoint == '/mnt/FileStore/MountFolder/' for mount in dbutils.fs.mounts()):
  try:
    dbutils.fs.mount(
    source = "wasbs://{}@{}.blob.core.windows.net".format(blobContainerName, storageAccountName),
    mount_point = "/mnt/FileStore/MountFolder/",
    extra_configs = {'fs.azure.account.key.' + storageAccountName + '.blob.core.windows.net': storageAccountAccessKey}
  )
  except Exception as e:
    print("already mounted. Try to unmount first")

display(dbutils.fs.ls("dbfs:/mnt/FileStore/MountFolder"))
# os.listdir('/dbfs/mnt/FileStore/MountFolder/')

path,name,size
dbfs:/mnt/FileStore/MountFolder/nasdaq_20200805.txt,nasdaq_20200805.txt,75600
dbfs:/mnt/FileStore/MountFolder/nasdaq_20200806.txt,nasdaq_20200806.txt,75613
dbfs:/mnt/FileStore/MountFolder/nyse_20200805.txt,nyse_20200805.txt,34241
dbfs:/mnt/FileStore/MountFolder/nyse_20200806.txt,nyse_20200806.txt,34241


In [0]:
# read 4 files as RDDs
raw_nyse_20200805 = spark.sparkContext.textFile('wasbs://container1@saderekguidedcapstone.blob.core.windows.net/nyse_20200805.txt')
raw_nyse_20200806 = spark.sparkContext.textFile('wasbs://container1@saderekguidedcapstone.blob.core.windows.net/nyse_20200806.txt')
raw_nasdaq_20200805 = spark.sparkContext.textFile('wasbs://container1@saderekguidedcapstone.blob.core.windows.net/nasdaq_20200805.txt')
raw_nasdaq_20200806 = spark.sparkContext.textFile('wasbs://container1@saderekguidedcapstone.blob.core.windows.net/nasdaq_20200806.txt')

In [0]:
# this formats tuple from csv
def parse_csv(line):
    record_type_pos = 2
    record = line.split(',')
    try:
        # logic to parse records
        if record[record_type_pos] == 'T':
            event = (
                datetime.strptime(record[0], '%Y-%m-%d'), #trade_dt
                record[2], #rec_type
                record[3], #symbol
                record[6], #exchange
                datetime.strptime(record[4], '%Y-%m-%d %H:%M:%S.%f'), #event_tm
                int(record[5]), #event_seq_nb
                datetime.strptime(record[1], '%Y-%m-%d %H:%M:%S.%f'), #arrival_tm
                float(record[7]), #trade_pr
                int(record[8]), #trade_size
                None, #bid_pr
                None, #bid_size
                None, #ask_pr
                None, #ask_size
                'T', #partition
                '' #list line if bad data
            )
            return event
        elif record[record_type_pos] == 'Q':
            event = (
                datetime.strptime(record[0], '%Y-%m-%d'), #trade_dt
                record[2], #rec_type
                record[3], #symbol
                record[6], #exchange
                datetime.strptime(record[4], '%Y-%m-%d %H:%M:%S.%f'), #event_tm
                int(record[5]), #event_seq_nb
                datetime.strptime(record[1], '%Y-%m-%d %H:%M:%S.%f'), #arrival_tm
                None, #trade_pr
                None, #trade_size
                float(record[7]), #bid_pr
                int(record[8]), #bid_size
                float(record[9]), #ask_pr
                int(record[10]), #ask_size
                'Q', #partition
                '' #list line if bad data
            )
            return event
    except Exception as e:
        # save record to dummy event in bad partition
        event = (
            None, # trade_dt
            '', # rec_type
            '', # symbol
            '', # exchange
            None, # event_tm
            None, # event_seq_nb
            None, #arrival_tm
            None, #trade_pr
            None, #trade_size
            None, # bid_pr
            None, # bid_size
            None, # ask_pr
            None, # ask_size
            'B', # partition
            line # list line if bad data
        )
        return event

In [0]:
# this formats tuple from json
def parse_json(line):
    record = json.loads(line)
    record_type = record['event_type']
    try:
        # logic to parse records
        if record_type == 'T':
            event = (
                datetime.strptime(record['trade_dt'], '%Y-%m-%d'), #trade_dt
                record_type, #rec_type
                record['symbol'], #symbol
                record['exchange'], #exchange
                datetime.strptime(record['event_tm'], '%Y-%m-%d %H:%M:%S.%f'), #event_tm
                int(record['event_seq_nb']), #event_seq_nb
                datetime.strptime(record['file_tm'], '%Y-%m-%d %H:%M:%S.%f'), #arrival_tm
                float(record['price']), #trade_pr
                int(record['size']), #trade_size
                None, #bid_pr
                None, #bid_size
                None, #ask_pr
                None, #ask_size
                'T', #partition
                '' #list line if bad data
            )
            return event
        elif record_type == 'Q':
            event = (
                datetime.strptime(record['trade_dt'], '%Y-%m-%d'), #trade_dt
                record_type, #rec_type
                record['symbol'], #symbol
                record['exchange'], #exchange
                datetime.strptime(record['event_tm'], '%Y-%m-%d %H:%M:%S.%f'), #event_tm
                int(record['event_seq_nb']), #event_seq_nb
                datetime.strptime(record['file_tm'], '%Y-%m-%d %H:%M:%S.%f'), #arrival_tm
                None, #trade_pr
                None, #trade_size
                float(record['bid_pr']), #bid_pr
                int(record['bid_size']), #bid_size
                float(record['ask_pr']), #ask_pr
                int(record['ask_size']), #ask_size
                'Q', #partition
                '' #list line if bad data
            )
            return event
    except Exception as e:
        # save record to dummy event in bad partition
        event = (
            None, # trade_dt
            '', # rec_type
            '', # symbol
            '', # exchange
            None, # event_tm
            None, # event_seq_nb
            None, #arrival_tm
            None, #trade_pr
            None, #trade_size
            None, # bid_pr
            None, # bid_size
            None, # ask_pr
            None, # ask_size
            'B', # partition
            line # list line if bad data
        )
        return event

In [0]:
# dataframe schema to impose
schema = StructType([
  StructField('trade_dt', DateType()),
  StructField('rec_type', StringType()),
  StructField('symbol', StringType()),
  StructField('exchange', StringType()),
  StructField('event_tm', TimestampType()),
  StructField('event_seq_nb', IntegerType()),
  StructField('arrival_tm', TimestampType()),
  StructField('trade_pr', FloatType()),
  StructField('trade_size', IntegerType()),
  StructField('bid_pr', FloatType()),
  StructField('bid_size', IntegerType()),
  StructField('ask_pr', FloatType()),
  StructField('ask_size', IntegerType()),
  StructField('partition', StringType()),
  StructField('bad_data', StringType())
])

# format RDDs
parsed_nyse_20200805 = raw_nyse_20200805.map(lambda line: parse_csv(line))
parsed_nyse_20200806 = raw_nyse_20200806.map(lambda line: parse_csv(line))
parsed_nasdaq_20200805 = raw_nasdaq_20200805.map(lambda line: parse_json(line))
parsed_nasdaq_20200806 = raw_nasdaq_20200806.map(lambda line: parse_json(line))

# create dataframes with schemas
data_nyse_20200805 = spark.createDataFrame(parsed_nyse_20200805, schema=schema)
data_nyse_20200806 = spark.createDataFrame(parsed_nyse_20200806, schema=schema)
data_nasdaq_20200805 = spark.createDataFrame(parsed_nasdaq_20200805, schema=schema)
data_nasdaq_20200806 = spark.createDataFrame(parsed_nasdaq_20200806, schema=schema)

# example of dataframe
data_nasdaq_20200806.printSchema()
data_nasdaq_20200806.show()

root
-- trade_dt: date (nullable = true)
-- rec_type: string (nullable = true)
-- symbol: string (nullable = true)
-- exchange: string (nullable = true)
-- event_tm: timestamp (nullable = true)
-- event_seq_nb: integer (nullable = true)
-- arrival_tm: timestamp (nullable = true)
-- trade_pr: float (nullable = true)
-- trade_size: integer (nullable = true)
-- bid_pr: float (nullable = true)
-- bid_size: integer (nullable = true)
-- ask_pr: float (nullable = true)
-- ask_size: integer (nullable = true)
-- partition: string (nullable = true)
-- bad_data: string (nullable = true)

+----------+--------+------+--------+--------------------+------------+-------------------+--------+----------+---------+--------+---------+--------+---------+--------+
 trade_dt|rec_type|symbol|exchange| event_tm|event_seq_nb| arrival_tm|trade_pr|trade_size| bid_pr|bid_size| ask_pr|ask_size|partition|bad_data|
+----------+--------+------+--------+--------------------+------------+-------------------+--------+----------+---------+--------+---------+--------+---------+--------+
2020-08-06| Q| SYMA| NASDAQ|2020-08-06 09:38:...| 1|2020-08-06 09:30:00| null| null|78.133705| 100|79.825165| 100| Q| |
2020-08-06| Q| SYMA| NASDAQ|2020-08-06 09:46:...| 2|2020-08-06 09:30:00| null| null| 76.52305| 100| 76.57241| 100| Q| |
2020-08-06| Q| SYMA| NASDAQ|2020-08-06 09:52:...| 3|2020-08-06 09:30:00| null| null| 78.74535| 100| 79.0928| 100| Q| |
2020-08-06| Q| SYMA| NASDAQ|2020-08-06 09:58:...| 4|2020-08-06 09:30:00| null| null|75.613625| 100|76.949776| 100| Q| |
2020-08-06| Q| SYMA| NASDAQ|2020-08-06 10:07:...| 5|2020-08-06 09:30:00| null| null| 77.45084| 100|78.725334| 100| Q| |
2020-08-06| Q| SYMA| NASDAQ|2020-08-06 10:15:...| 6|2020-08-06 09:30:00| null| null| 79.29842| 100|81.071915| 100| Q| |
2020-08-06| Q| SYMA| NASDAQ|2020-08-06 10:22:...| 7|2020-08-06 09:30:00| null| null| 77.76145| 100| 79.24523| 100| Q| |
2020-08-06| Q| SYMA| NASDAQ|2020-08-06 10:29:...| 8|2020-08-06 09:30:00| null| null|75.601135| 100| 76.95535| 100| Q| |
2020-08-06| Q| SYMA| NASDAQ|2020-08-06 10:35:...| 9|2020-08-06 09:30:00| null| null| 76.30005| 100|77.618706| 100| Q| |
2020-08-06| T| SYMA| NASDAQ|2020-08-06 10:42:...| 10|2020-08-06 09:30:00|78.93246| 368| null| null| null| null| T| |
2020-08-06| Q| SYMA| NASDAQ|2020-08-06 10:50:...| 11|2020-08-06 09:30:00| null| null| 77.96207| 100| 79.18113| 100| Q| |
2020-08-06| Q| SYMA| NASDAQ|2020-08-06 10:59:...| 12|2020-08-06 09:30:00| null| null|76.279816| 100| 77.32987| 100| Q| |
2020-08-06| Q| SYMA| NASDAQ|2020-08-06 11:06:...| 13|2020-08-06 09:30:00| null| null| 78.76234| 100| 79.92418| 100| Q| |
2020-08-06| Q| SYMA| NASDAQ|2020-08-06 11:15:...| 14|2020-08-06 09:30:00| null| null|79.196526| 100|80.117584| 100| Q| |
2020-08-06| Q| SYMA| NASDAQ|2020-08-06 11:23:...| 15|2020-08-06 09:30:00| null| null| 74.83831| 100|76.313354| 100| Q| |
2020-08-06| Q| SYMA| NASDAQ|2020-08-06 11:32:...| 16|2020-08-06 09:30:00| null| null| 76.65085| 100| 77.56526| 100| Q| |
2020-08-06| Q| SYMA| NASDAQ|2020-08-06 11:40:...| 17|2020-08-06 09:30:00| null| null|75.345146| 100| 76.79113| 100| Q| |
2020-08-06| Q| SYMA| NASDAQ|2020-08-06 11:49:...| 18|2020-08-06 09:30:00| null| null| 75.37165| 100|76.572685| 100| Q| |
2020-08-06| Q| SYMA| NASDAQ|2020-08-06 11:55:...| 19|2020-08-06 09:30:00| null| null| 75.16338| 100|75.755356| 100| Q| |
2020-08-06| T| SYMA| NASDAQ|2020-08-06 12:00:...| 20|2020-08-06 09:30:00| 77.0967| 51| null| null| null| null| T| |
+----------+--------+------+--------+--------------------+------------+-------------------+--------+----------+---------+--------+---------+--------+---------+--------+
only showing top 20 rows

In [0]:
# write parquet files in databricks cluster
data_nyse_20200805.write.partitionBy('partition').mode('overwrite').parquet('output_dir')
data_nyse_20200806.write.partitionBy('partition').mode('overwrite').parquet('output_dir')
data_nasdaq_20200805.write.partitionBy('partition').mode('overwrite').parquet('output_dir')
data_nasdaq_20200806.write.partitionBy('partition').mode('overwrite').parquet('output_dir')

In [0]:
# check locations of parquet files
# os.listdir('/dbfs/output_dir/')
os.listdir('/dbfs/output_dir/partition=Q/')

Out[13]: ['_committed_1219997597174865034',
 '_committed_293078419103904347',
 '_committed_3165818644618988306',
 '_committed_3168304201065439625',
 '_committed_3552474771337509288',
 '_committed_3559751507643289125',
 '_committed_4969766954095086422',
 '_committed_4975854336944468212',
 '_committed_5397892984581443597',
 '_committed_6010572577373454651',
 '_committed_727903487361347123',
 '_committed_9222919642996845500',
 '_started_1219997597174865034',
 '_started_3165818644618988306',
 '_started_3168304201065439625',
 '_started_3552474771337509288',
 '_started_3559751507643289125',
 '_started_4969766954095086422',
 '_started_4975854336944468212',
 '_started_6010572577373454651',
 '_started_727903487361347123',
 '_started_9222919642996845500',
 'part-00000-tid-4975854336944468212-af0f2054-cd3d-49af-8308-7fb5f235984e-15-1.c000.snappy.parquet',
 'part-00001-tid-4975854336944468212-af0f2054-cd3d-49af-8308-7fb5f235984e-16-1.c000.snappy.parquet']